In [1]:
import os

In [2]:
%pwd


'e:\\Projects\\Kidney_Disease_Classification\\research'

In [3]:
os.chdir("../")


In [7]:
%pwd

'e:\\Projects\\Kidney_Disease_Classification'

In [5]:
from src.cnnClassifier import config


In [6]:
import yaml

with open('config\config.yaml', 'r') as file:
    config_data = yaml.safe_load(file)


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [9]:
from cnnClassifier.constants import *

In [10]:
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
import tensorflow as tf

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [33]:
from src.cnnClassifier.config.configuration import ConfigurationManager


In [13]:
import os
import urllib.request as request
from zipfile import ZipFile
import time

In [14]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [22]:
config_data = {
    'artifacts_root': 'artifacts', 
    'data_ingestion': {
        'root_dir': 'artifacts/data_ingestion', 
        'source_URL': 'https://drive.google.com/file/d/1YhKFP8rX6afx5gPFRux2kSXv9KEs6un7/view?usp=sharing', 
        'local_data_file': 'artifacts/data_ingestion/data.zip', 
        'unzip_dir': 'artifacts/data_ingestion'
    }
}


In [21]:
print(config_data['artifacts_root'])  # prints 'artifacts'

print(config_data['data_ingestion']['source_URL'])  # prints the source URL


artifacts
https://drive.google.com/file/d/1YhKFP8rX6afx5gPFRux2kSXv9KEs6un7/view?usp=sharing


In [20]:
import yaml

config_filepath = 'config/config.yaml'

try:
    with open(config_filepath, 'r') as file:
        config_data = yaml.safe_load(file)
        print("Configuration loaded successfully!")
except FileNotFoundError:
    print(f"Config file '{config_filepath}' not found.")
except yaml.YAMLError as e:
    print(f"Error loading YAML file: {e}")


Configuration loaded successfully!


In [19]:

# Print the entire loaded configuration
print(config_data)

{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1YhKFP8rX6afx5gPFRux2kSXv9KEs6un7/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}}


In [18]:
# Assuming config_data is already loaded from the previous step

try:
    training_config = config_data['training']
    print("Found 'training' section in the configuration.")
    print("Training configuration:")
    print(training_config)
except KeyError:
    print("Error: 'training' section not found in the configuration.")
    # Add additional error handling or raise an exception as needed


Error: 'training' section not found in the configuration.


In [15]:
config.get_training_config() # To check the type of the returned object

AttributeError: module 'src.cnnClassifier.config' has no attribute 'get_training_config'

In [23]:
# Assuming config_data is already loaded from the previous step

# Print the entire loaded configuration
print(config_data)

# Access specific sections of the configuration
print(config_data['data_ingestion'])
print(config_data['prepare_base_model'])
print(config_data['training'])



{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1YhKFP8rX6afx5gPFRux2kSXv9KEs6un7/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
{'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1YhKFP8rX6afx5gPFRux2kSXv9KEs6un7/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}


KeyError: 'prepare_base_model'

In [25]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-05-14 11:36:44,782: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-14 11:36:44,789: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-14 11:36:44,792: INFO: common: created directory at: artifacts]
[2024-05-14 11:36:44,795: INFO: common: created directory at: artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
23/23 [==============================] - 116s 5s/step - loss: 9.5613 - accuracy: 0.6180 - val_loss: 10.5218 - val_accuracy: 0.4375
